In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('hfd20-21data.txt')
winter2 = pd.read_csv ('hfd21-22data.txt')
winter3 = pd.read_csv ('hfd22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
hfd = LocationInfo(
    'Hartford', 'Connecticut', 
    latitude=41.7658,  
    longitude=-72.6734,
    timezone='America/Connecticut'
)

hfd

LocationInfo(name='Hartford', region='Connecticut', timezone='America/Connecticut', latitude=41.7658, longitude=-72.6734)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(hfd.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(hfd.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(hfd.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:14  2020-12-01 16:20:27  2020-12-01
1   2020-12-02 07:00:16  2020-12-02 16:20:13  2020-12-02
2   2020-12-03 07:01:16  2020-12-03 16:20:01  2020-12-03
3   2020-12-04 07:02:16  2020-12-04 16:19:51  2020-12-04
4   2020-12-05 07:03:14  2020-12-05 16:19:43  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:33:05  2021-02-24 17:35:10  2021-02-24
86  2021-02-25 06:31:34  2021-02-25 17:36:22  2021-02-25
87  2021-02-26 06:30:02  2021-02-26 17:37:35  2021-02-26
88  2021-02-27 06:28:28  2021-02-27 17:38:47  2021-02-27
89  2021-02-28 06:26:55  2021-02-28 17:39:58  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:58:59  2021-12-01 16:20:31  2021-12-01
1   2021-12-02 07:00:01  2021-12-02 16:20:16  2021-12-02
2   2021-12-03 07:01:02  2021-12-03 16:20:03  2021-12-03
3   2021-12-04 07:02:02  2021-12-04 16:19:53  2021-12-04
4   2021-12-05 07:03:00  2021-12-05 16:19:45  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:33:27  2022-02-24 17:34:52  2022-02-24
86  2022-02-25 06:31:56  2022-02-25 17:36:05  2022-02-25
87  2022-02-26 06:30:24  2022-02-26 17:37:17  2022-02-26
88  2022-02-27 06:28:51  2022-02-27 17:38:29  2022-02-27
89  2022-02-28 06:27:17  2022-02-28 17:39:41  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:58:44  2022-12-01 16:20:35  2022-12-01
1   2022-12-02 06:59:46  2022-12-02 16:20:19  2022-12-02
2   2022-12-03 07:00:47  2022-12-03 16:20:06  2022-12-03
3   2022-12-04 07:01:47  2022-12-04 16:19:55  2022-12-04
4   2022-12-05 07:02:46  2022-12-05 16:19:46  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:33:49  2023-02-24 17:34:35  2023-02-24
86  2023-02-25 06:32:18  2023-02-25 17:35:47  2023-02-25
87  2023-02-26 06:30:46  2023-02-26 17:37:00  2023-02-26
88  2023-02-27 06:29:14  2023-02-27 17:38:12  2023-02-27
89  2023-02-28 06:27:40  2023-02-28 17:39:23  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HFD,2020-12-01 00:53:00,0.7600,-RA,2020-12-01,2020-12-01 06:59:14,2020-12-01 16:20:27
1,HFD,2020-12-01 02:53:00,0.2500,-RA,2020-12-01,2020-12-01 06:59:14,2020-12-01 16:20:27
2,HFD,2020-12-04 23:53:00,0.2500,-RA,2020-12-04,2020-12-04 07:02:16,2020-12-04 16:19:51
3,HFD,2020-12-05 00:53:00,1.0200,-RA,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
4,HFD,2020-12-05 01:53:00,0.7600,-RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
...,...,...,...,...,...,...,...
219,HFD,2021-02-27 13:53:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
220,HFD,2021-02-27 14:53:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
221,HFD,2021-02-27 15:53:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
222,HFD,2021-02-27 16:53:00,1.2700,RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HFD,2021-12-02 07:53:00,0.0001,-RA,2021-12-02,2021-12-02 07:00:01,2021-12-02 16:20:16
1,HFD,2021-12-02 08:53:00,0.2500,-RA,2021-12-02,2021-12-02 07:00:01,2021-12-02 16:20:16
2,HFD,2021-12-02 09:53:00,0.5100,-RA,2021-12-02,2021-12-02 07:00:01,2021-12-02 16:20:16
3,HFD,2021-12-02 10:53:00,0.0001,-RA,2021-12-02,2021-12-02 07:00:01,2021-12-02 16:20:16
4,HFD,2021-12-02 19:53:00,0.0001,-RA,2021-12-02,2021-12-02 07:00:01,2021-12-02 16:20:16
...,...,...,...,...,...,...,...
239,HFD,2022-02-25 13:53:00,2.2900,-SN BR,2022-02-25,2022-02-25 06:31:56,2022-02-25 17:36:05
240,HFD,2022-02-25 14:53:00,2.5400,-SNPL BR,2022-02-25,2022-02-25 06:31:56,2022-02-25 17:36:05
241,HFD,2022-02-25 17:53:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:31:56,2022-02-25 17:36:05
242,HFD,2022-02-25 18:53:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:31:56,2022-02-25 17:36:05


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HFD,2022-12-03 13:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:00:47,2022-12-03 16:20:06
1,HFD,2022-12-03 14:53:00,0.7600,-RA,2022-12-03,2022-12-03 07:00:47,2022-12-03 16:20:06
2,HFD,2022-12-03 15:53:00,1.5200,RA,2022-12-03,2022-12-03 07:00:47,2022-12-03 16:20:06
3,HFD,2022-12-03 16:53:00,2.5400,RA BR,2022-12-03,2022-12-03 07:00:47,2022-12-03 16:20:06
4,HFD,2022-12-03 17:53:00,1.7800,RA BR,2022-12-03,2022-12-03 07:00:47,2022-12-03 16:20:06
...,...,...,...,...,...,...,...
215,HFD,2023-02-24 06:53:00,0.2500,-RA,2023-02-24,2023-02-24 06:33:49,2023-02-24 17:34:35
216,HFD,2023-02-25 15:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:32:18,2023-02-25 17:35:47
217,HFD,2023-02-25 16:53:00,0.0001,SN,2023-02-25,2023-02-25 06:32:18,2023-02-25 17:35:47
218,HFD,2023-02-25 17:53:00,0.5100,-SN,2023-02-25,2023-02-25 06:32:18,2023-02-25 17:35:47


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
9,HFD,2020-12-05 07:53:00,0.2500,-RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
10,HFD,2020-12-05 08:53:00,0.7600,-RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
11,HFD,2020-12-05 09:53:00,2.0300,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
12,HFD,2020-12-05 10:53:00,3.8100,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
13,HFD,2020-12-05 11:53:00,3.8100,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
...,...,...,...,...,...,...,...
218,HFD,2021-02-27 10:53:00,0.0001,-SN,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
219,HFD,2021-02-27 13:53:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
220,HFD,2021-02-27 14:53:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47
221,HFD,2021-02-27 15:53:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
10,HFD,2020-12-05 08:53:00,0.7600,-RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43,2020-12-05 08:03:14,2020-12-05 15:19:43
11,HFD,2020-12-05 09:53:00,2.0300,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43,2020-12-05 08:03:14,2020-12-05 15:19:43
12,HFD,2020-12-05 10:53:00,3.8100,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43,2020-12-05 08:03:14,2020-12-05 15:19:43
13,HFD,2020-12-05 11:53:00,3.8100,RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43,2020-12-05 08:03:14,2020-12-05 15:19:43
14,HFD,2020-12-05 12:53:00,6.8600,+RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43,2020-12-05 08:03:14,2020-12-05 15:19:43
...,...,...,...,...,...,...,...,...,...
217,HFD,2021-02-27 09:53:00,0.0001,-SN,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47,2021-02-27 07:28:28,2021-02-27 16:38:47
218,HFD,2021-02-27 10:53:00,0.0001,-SN,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47,2021-02-27 07:28:28,2021-02-27 16:38:47
219,HFD,2021-02-27 13:53:00,0.0001,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47,2021-02-27 07:28:28,2021-02-27 16:38:47
220,HFD,2021-02-27 14:53:00,0.5100,-RA BR,2021-02-27,2021-02-27 06:28:28,2021-02-27 17:38:47,2021-02-27 07:28:28,2021-02-27 16:38:47


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.5862766233766235
2021-2022: 0.8549789473684212
2022-2023: 0.8976034482758621


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HFD,2020-12-01 00:53:00,0.7600,-RA,2020-12-01,2020-12-01 06:59:14,2020-12-01 16:20:27
1,HFD,2020-12-01 02:53:00,0.2500,-RA,2020-12-01,2020-12-01 06:59:14,2020-12-01 16:20:27
2,HFD,2020-12-04 23:53:00,0.2500,-RA,2020-12-04,2020-12-04 07:02:16,2020-12-04 16:19:51
3,HFD,2020-12-05 00:53:00,1.0200,-RA,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
4,HFD,2020-12-05 01:53:00,0.7600,-RA BR,2020-12-05,2020-12-05 07:03:14,2020-12-05 16:19:43
...,...,...,...,...,...,...,...
213,HFD,2021-02-22 21:53:00,2.7900,RA BR,2021-02-22,2021-02-22 06:36:06,2021-02-22 17:32:44
214,HFD,2021-02-22 22:53:00,1.0200,-RA BR,2021-02-22,2021-02-22 06:36:06,2021-02-22 17:32:44
215,HFD,2021-02-23 18:53:00,0.0001,-RA,2021-02-23,2021-02-23 06:34:36,2021-02-23 17:33:57
216,HFD,2021-02-25 04:53:00,0.0001,-RA,2021-02-25,2021-02-25 06:31:34,2021-02-25 17:36:22


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_16204\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7684088235294118
2021-2022: 0.5295122807017544
2022-2023: 0.6687507936507937
